In [1]:
from google.cloud import bigquery
client = bigquery.Client()
# Which countries do not use ppm as a unit to measure pollutants?
query = """SELECT DISTINCT country
            FROM `bigquery-public-data.openaq.global_air_quality`
            WHERE unit != 'ppm'
        """
query_job = client.query(query)
rows = list(query_job.result(timeout=30))
print(len(rows), "results")
print([x.country for x in rows])

64 results
['US', 'FR', 'TH', 'CL', 'IN', 'NL', 'GB', 'CH', 'PT', 'ES', 'BR', 'TR', 'PE', 'NO', 'HK', 'LV', 'IT', 'CN', 'GH', 'PL', 'SI', 'BD', 'MT', 'VN', 'BA', 'BE', 'TW', 'LT', 'DE', 'CO', 'SE', 'NG', 'MK', 'AU', 'BH', 'PH', 'RU', 'CA', 'AD', 'IE', 'AT', 'ID', 'HU', 'LK', 'UG', 'FI', 'IL', 'LU', 'HR', 'XK', 'AE', 'NP', 'SG', 'CZ', 'MX', 'DK', 'ET', 'KW', 'MN', 'AR', 'RS', 'ZA', 'SK', 'GI']


In [10]:
# if DISTINCT is not used
unique_res = {}
for x in rows:
    if x.country in unique_res:
        unique_res[x.country] += 1
    else:
        unique_res[x.country] = 1
print(unique_res.keys())

dict_keys(['LT', 'MK', 'ET', 'RU', 'BH', 'CN', 'RS', 'NG', 'IE', 'AE', 'SE', 'CH', 'HR', 'AR', 'PL', 'XK', 'US', 'PH', 'AD', 'NP', 'IL', 'CO', 'BR', 'IT', 'MN', 'PT', 'MX', 'DK', 'LV', 'CL', 'GH', 'GB', 'KW', 'FI', 'PE', 'TW', 'BA', 'CZ', 'HU', 'IN', 'AU', 'AT', 'NO', 'BE', 'DE', 'ID', 'ES', 'NL', 'FR', 'TR', 'ZA', 'LU', 'VN', 'SI', 'SG', 'MT', 'LK', 'HK', 'TH', 'UG', 'BD', 'SK', 'CA', 'GI'])


In [12]:
# Which pollutants have a value of exactly 0, in what locations?
query =  """SELECT pollutant, location
            FROM `bigquery-public-data.openaq.global_air_quality`
            WHERE value = 0.00"""
query_job = client.query(query)
rows = list(query_job.result(timeout=30))
# print(len(rows), "results")
# print([x.pollutant for x in rows])

import pandas as pd
df = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
# e.g. places where there is a recorded CO level of 0.
print(df['location'][df['pollutant'] == 'co'])

1                                        Sur
2                          Akron - NIHF_STEM
3                                     Lee HS
4                                    MT00005
5                                      Puren
6                                      Rengo
7                                    REIBOLD
8                                     Indura
9                                       МАДИ
10                                 Саларьево
11                                    Osorno
12                                     Brist
13                                  Alsancak
14                         Wels Linzerstraße
15                                   ES1601A
16                  Din Daeng Rd., Din Daeng
17                               Lota urbana
18                                   Quilpue
19                                   Romeral
20                                   LT00001
21                                   LT00003
22                              PARIS Centre
23        

In [26]:
# get the schema to explain what each column means
table_ref = client.dataset('openaq', project='bigquery-public-data').table('global_air_quality')
for name, desc in [(s.name, s.description) for s in client.get_table(table_ref).schema]:
    print(name, ":", desc)

location : Location where data was measured
city : City containing location
country : Country containing measurement in 2 letter ISO code
pollutant : Name of the Pollutant being measured. Allowed values: PM25, PM10, SO2, NO2, O3, CO, BC
value : Latest measured value for the pollutant
timestamp : The datetime at which the pollutant was measured, in ISO 8601 format
unit : The unit the value was measured in coded by UCUM Code
source_name : Name of the source of the data
latitude : Latitude in decimal degrees. Precision >3 decimal points.
longitude : Longitude in decimal degrees. Precision >3 decimal points.
averaged_over_in_hours : The number of hours the value was averaged over.
